# Basic Gitlet Objects

Gitlet stores 3 main types of objects:
1. BLOB objects - the actual file contents
2. TREE objects - directory structure
3. COMMIT objects - metadata about the commit

### BLOB objects
- For a python script, the BLOB object is the text of your script converted to bytes

### TREE objects
- The files, names, and hashes
- It is stored as a pickled dictionary, i.e. bytes representing your directory structure

### COMMIT objects
- Metadata about the commit
  - Message, author, timestamp, tree (where in the tree is this commit), parent
  - It is stored as a pickled commit dictionary, i.e. bytes representing the dictionary that contains your commit metadata

# Hashes

## Hash Tables

**MOST IMPORTANT THING TO KNOW ABOUT HASH TABLES**

If you glean one thing from this, know: **hash tables are good because they generally have a constant lookup time.** i.e. no matter how big the list is, the time to get an index (which will get you the value of a key) is constant.

*Hash tables are different from cryptographic hashes like SHA-1*

What are hashes? A hash is number that you get after putting a key into a hash function. The number is an idex. THe purpose is for fast lookups: isntead of searching through a whole row in an array, just jump to the correct spot.
- In this example I use hashes like 829 as an example but normally they're long, like `ef92b778bafe771e89245b89ecbc08a44a4e166c06659911881f383d4473e94f`

### How do I hash key-value pairs?

**Step 0**

You have an array with key-value pairs:

| **Key** | "b" | "a" | "c" |
| - | - | - | - | 
| **Value** | "banana" | "apple" | "cantaloupe" | 

You see "a" has index 0 and index 0 of **Value** corresponds to "apple", and so forth for the other **Keys**.

**Step 1** 

Create a hash table (a really big array)

**Step 2**

Convert the **Keys** to bytes, i.e. **serialize** it.

Put the byte representatino of the **Keys** into a hash function that spits out a number
- Hash functions spit out numbers that are irreversible (you can't know what they key is given the hash function output) and independent of the other outputs (one output doesn't give you information about the other outputs)

Each output from the hash function is an **index**. For example, if "b" goes into the function and the output is 489, then "banana" will be stored at index 489 of the hash table.

| **Index** | ... | 489 | ... |
| - | - | - | - | 
| **Value** | ... | "banana" | ... | 

**Step 3**

Now you can quickly look up the value for a key! If you want the value for "b", put "b" into the hash function, i.e. hash("b") = 489 and then look for index 489 in your table and you will find "banana." (where you are actually putting bytes into the hash function)


### Collisions

What if two inputs (two keys) hash to the same output (same index)? E.g. hash("a") = 892 and hash("b") = 892.

You can chain, which means store multiple items at the same index, i.e. the 892nd index in your table contains a list `["apple", "banana"]`. Often you want some amount of chaining because this helps reduce your table size and saves memory. 

So yes you should minimize collisions in your hashing, but strategic collisions can be good because it reduces memory useage. 

## Side Note: Cryptographic Hashes

Instead of finding where to store data, crypotgraphic hashes are used to create an unforgeable fingerprint of data. 

Suppose a user create a password `"secretPass"` and we run a hash algorithm and get a hash `"abc123"`. (Not sure how our algorithm got us that but don't worry about it...)

In our database, instead of storing `"secretPass"`, we store the hash, `"abc123"`. We then forget about the password `"secretPass"`.

When the user logs in:

Enter password: `"secretPass"`

The password is hashed and if it is hashed to be `"abc123"` then the user is let in. But `"abc123"` won't hash to be `"abc123"` and the hash isn't reversible, so knowing the hash doesn't do anything for a hacker.

Therefore, if hackers steal the database, they only get hashes which doesn't do anything. 

For cryptographic hashes, hash functions **must be**:
- deterministic
- hugely affected by a tiny chance (e.g. `"Hello"` and `"hello"` result in very different hashes)
- irreversible



## Hashes in Context of Gitlet

For each commit, we run the commit dictionary object through a hash function to get a unique hash, which acts as a unique ID for the commit dictionary object. 

The idea of a **Value** doesn't matter because the hash itself is the identifier. We have a file containing a commit <--> file name contains the hash <--> the file iteself contains the commit content. So for gitlet, **hash == filename**.

# Branches

What is a branch you ask? Well you have come to the right place for I shall educate thee!

A branch is just a pointer (label) that points to a commit. "Master" is just a conventional name. Your `master` file (it's a file not a folder) is a text file containing your commit hash. That's it!

Everytime you make a new commit and use the master branch, the old master file gets overwritten with the new commit's hash so the old hash is gone from the master branch file.

BUT you are not losing history. All commits are stored in the `objects` file. Each commit knows its parents since we record the parent in the commit dictionary object. So, though `master` only points to the latest commit, we can use the parents to trace back all the way to the initial commit. 

A branch is called a branch because it can diverge from the main timeline. 

The master branch looks like:

|(initial commit)| --> |(commit 1)| --> |(commit 2)| --> |(commit 3)|
|-|-|-|-|-|-|-|

We can trace back to initial commit from commit 3.

But we might have a feature branch that looks like:

|(commit 2)| --> |(commit a)| --> |(commit b)|
|-|-|-|-|-|

It's off the main timeline. 


# `argparse`

*(Much of code/info from https://docs.python.org/3/library/argparse.html)*

Turns out parsing arguments is a common thing that people do. I was truly shocked to learn that.

`argparse` is a standard library for working with the command line. 

We can create an instance of `argparse.ArgumentParser()`. It has some argument specifications.

In [ ]:


parser = argparse.ArgumentParser(
                    prog='ProgramName',
                    description='What the program does',
                    epilog='Text at the bottom of help')


`ArgumentParser.add_argument()` attaches more arguments to the parser.

In [ ]:
parser.add_argument('filename')           # positional argument
parser.add_argument('-c', '--count')      # option that takes a value
parser.add_argument('-v', '--verbose',
                    action='store_true')  # on/off flag

`ArgumentParser.parse_args()` runs the parser. It extracts data.

In [ ]:
args = parser.parse_args()
print(args.filename, args.count, args.verbose)

Sometimes we want various parsers that all share some set of the same charactaristics while also having their own unique ones. Instead of repeated the shared arguments, we can have a single parser with all the shared arguments and passed to `parents=` argument. 

In [ ]:
parent_parser = argparse.ArgumentParser(add_help=False)
parent_parser.add_argument('--parent', type=int)

foo_parser = argparse.ArgumentParser(parents=[parent_parser])
foo_parser.add_argument('foo')
foo_parser.parse_args(['--parent', '2', 'XXX'])


bar_parser = argparse.ArgumentParser(parents=[parent_parser])
bar_parser.add_argument('--bar')
bar_parser.parse_args(['--bar', 'YYY'])

A super useful utility for us is `ArgumentParser.add_subparsers`. This helps one command also have a bunch of subcommands. One example where we might need this is when we want a command that adds something to the staging area, but we want to know if we want to add it to the `"additions"` part or the `"removals"` part. This would be only if we were structuring the staging area like that. At the time of writing this paragraph, I'm undecided.

In [ ]:
# create the top-level parser
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('--foo', action='store_true', help='foo help')
subparsers = parser.add_subparsers(help='subcommand help')

# create the parser for the "a" command
parser_a = subparsers.add_parser('a', help='a help')
parser_a.add_argument('bar', type=int, help='bar help')

# create the parser for the "b" command
parser_b = subparsers.add_parser('b', help='b help')
parser_b.add_argument('--baz', choices=('X', 'Y', 'Z'), help='baz help')

# parse some argument lists
parser.parse_args(['a', '12'])

parser.parse_args(['--foo', 'b', '--baz', 'Z'])

So let's contextualize a little bit. I have an init function. I want it to run when someone uses the command `init` in the command line. 

In [ ]:
import argparse

def init():
    print("Initializing a new repository.")

def main():
    parser = argparse.ArgumentParser(description="MiniGit CLI")
    subparsers = parser.add_subparsers(dest = "command")
    subparsers.add_parser("init", help = "Initialize a new repository")

    if args.command == "init":
        init()
    else:
        parser.print_help()

main()

# It won't run properly here but if I did:
# "python minigit.py init" in the command line, I would get the output:
# "Initializing a new repository."

`argparse` parses the arguments **according to the structure that you define.** When I said `dest="command"`, this means that whatever subcommand the user types in is stored in `args.command`.

# Using `os.walk()`

`os.walk('.')` is a generator that gives tuples for each directory. 

**Command:**
```
for root, dirs, files in os.walk("."):
    print(f"Root: {root}")
    print(f"Dirs: {dirs}")
    print(f"Files: {files}")
    print()
```

**Output:**
```
Root: .
Dirs: ['src']
Files: ['main.py', 'utils.py']

Root: ./src
Dirs: ['data']
Files: ['helper.py']

Root: ./src/data
Dirs: []
Files: ['config.json']
```

`os.walk("your directory")` visits each folder one at a time. It starts at the top and records the root. It then records the folders (directories). It also records all the files. It gives you one tuple *per directory it visits*. 

Each tuple has three parts. The first is the root, as a simple string. It's the current directory path to the directory it's making a tuple for. The second is a *list* of subdirectories in that directory. The third is a *list* of files in that directory. 

So it might look like:
`("user/projects/minigit/dir1/", ["dir1", "dir_in_dir1"], ["file1", "file2", "file3"])`

So when you do `for root, dir, file` --> it treats the first part of the tuple as your `"root"`, the second part as your `"dir"` and the third as your `"file"`. 

Note:

```
for root, dirs, files in os.walk("."):
    # Step 1: os.walk says "I'm in directory X"
    # Step 2: os.walk says "Here are the subdirs I PLAN to visit next"
    # Step 3: You can modify 'dirs' HERE to change the plan
    # Step 4: os.walk visits those subdirectories in the NEXT iterations
```

**`dirs` is like a to-do list for `os.walk`**

# Pickle
`f.read()` reads the bytes but doesn't deserialize.

`pickle.load(f)` reads and deserializes.

# When HEAD is detached

When HEAD is detached, it'll just have the hash. When it's not detached, it has the actual path to the master branch as a string.

- git log just starts wherever HEAD is pointing and walks backwards through the commit history
- 

test test

test2test2